### 예비창업 패키지

In [ ]:
import json
import re

from docx import Document
from docx.shared import Pt
from openai import OpenAI

from docx2pdf import convert

In [ ]:
client = OpenAI(api_key='YOUR_API_KEY')

In [ ]:
template = """
1. 항목 제목  
   1-1. 제목  
       1) 소제목  
          - 세부 항목 1  
          - 세부 항목 2  
          - 세부 항목 3  
       2) 소제목  
          - 세부 항목 1  
          - 세부 항목 2  
          - 세부 항목 3  

2. 항목 제목  
   2-1. 제목  
       1) 소제목  
          - 세부 항목 1  
          - 세부 항목 2  
          - 세부 항목 3  
       2) 소제목  
          - 세부 항목 1  
          - 세부 항목 2  
          - 세부 항목 3  
          """

In [ ]:
# 1. 문제인식(Problem)
template_test1 = """
1. 문제인식(Problem)

    1-1. 창업아이템 배경 및 필요성
        - 창업 아이템(제품, 서비스 등)을 개발(개선) 배경과 이를 뒷받침할 근거, 동기 등을 제시하여 작성하세요.
        - 국내·외 시장에서 발생하는 주요 문제점과 새로운 기술 도입 필요성을 설명하세요.
        - 산업 변화, 정책 변화, 소비자 니즈 변화 등을 작성하세요.
        - 배경 및 필요성에서 발견한 문제점과 해결방안, 필요성, 제품·서비스를 개발하려는 목적과 사업을 통해 이루고자 하는 목표를 작성해주세요.
    1-2. 창업아이템 목표시장(고객) 설정 및 요구사항 분석
        - 창업 아이템 배경 및 필요성에 따라 정의된 시장(고객)에 대해 제공할 혜택과 행위를 제공할 시장(고객)을 설정하고 구체화하여 작성하세요.
        - 진출하려는 시장의 규모, 상황 및 특성, 경쟁 강도, 향후 전망, 고객 특성을 작성하세요.
        - 정의된 목표시장 요구사항에 대한 조사를 하고 분석 결과 및 문제의 객관적 근거들을 작성하세요."""
        
# 2. 실현가능성(Solution)
template_test2 = """
2. 실현가능성(Solution) (실현가능성 이외에는 작성 금지.)

    2-1. 창업 아이템 현황 및 준비 정도
        - 제품·서비스의 필요성에 대한 문제를 인식하고, 해당 제품·서비스 개발·개선을 위해 본 사업에 신청하기 이전까지 기획, 추진한 이력 등에 대해 작성하세요.
        - 사업 신청 시점의 아이템 개발 단계별 현황, 목표한 시장 및 고객의 반응, 현재까지의 주요 사업화 성과 등 전반적인 현황을 작성하세요.
    2-2. 창업 아이템 실현 및 구체화 방안
        - 협약 기간 내 실현 및 구체화하고자 하는 개선 방안을 작성해주세요.
        - 목표시장 요구사항 등 분석을 통해 파악된 문제점 및 개선점에 대해 핵심 기능·성능, 디자인 개선, 기타 사업화 활동을 실현 및 구체적인 개선 방안을 작성하세요.
        - 경쟁사 제품·서비스와의 비교를 통해 파악된 문제점에 대해 자사의 보유역량을 기반으로 경쟁력, 차별성 등을 확보하기 위한 방안을 작성하세요.
        - 기타 개선·구체화하고자 하는 제품·서비스 관련 기술 보호 계획을 작성하세요."""

# 3.성장전략(Scale-up)
template_test3 = """
3. 성장전략(Scale-up)

    3-1. 비즈니스 모델 및 사업화 추친 성과
        - 제품·서비스 비즈니스 모델 중 수익창출을 위한 수익 모델을 작성하세요.
        - 현재 진입한 목표시장(고객)을 진입 현황과 그간 사업화 추진성과(매출, 투자, 고용 등)를 작성하세요.
            (예시) 생산·출시, 홍보·마케팅 및 유통·판매 현황과 매출 실적 등 작성하세요.
        - 서비스의 경우 회원(이용자) 확보 및 다운로드 수 등 작성하세요.    
    3-2. 아이템 시장 확대 등 사업화 전략
        - 목표시장 내 세부 고객, 경쟁사, 시장 내 대체하고자 하는 제품, 서비스의 주요 정보 분석등을 통해 국내외 시장 확대 전략 수립을 작성하세요.
        - 목표시장에 진출 및 확대하기 위한 구체적인 방안과 고객 확보 전략, 수익 확대 전략 등을 작성하세요.
        - 협약기간 내 달성하고자 하는 성과(매출, 투자, 고용 등)을 기재하세요.
    3-3. 사업추진 일정 및 자금운용 계획
        - 사업 전체 로드맵(일정)
            예시)
            |순 번|  추진 내용 |       추진 기간       | 세부 내용 |
            |-----|-----------|----------------------|----------|
            |  1  | 시제품 제작|  2025.02 ~ 2025.04   |시제품 설계|
            
        <사업추진 일정(전체 사업단계)>
        예시)
        |순 번|          추진 내용         |       추진 기간        |         세부 내용         |
        |-----|---------------------------|-----------------------|--------------------------|
        |  1  |시장 조사 및 초기 마케팅 활동|   2025.04 ~ 2025.06   |SNS 캠페인, 온라인 광고 진행|
            
        - 협약기간 내 목표 및 달성 방안
            - 협약기간 내 추진하고자 하는 주요 내용과 목표, 일정, 최종결과물 등을 명확히 작성하세요. 
            
            <사업추진 일정(협약기간 내)>
            예시)
            |순번|     추진 내용   |      추진 기간      |      세부 내용      | 
            |----|----------------|--------------------|--------------------|
            |  1 | 프로토타입 개발 |  2025.04 ~ 2025.06 | 기술인략을 통한 개발 |
            
        - 정부지원사업비 집행계획 
            - 사업의 협약기간 동안 사업화를 수행하기 위해 필요한 필요 예산 및 계획서를 작성하세요.

        <사업비 집행 계획>
        *"정부지원사업비는 총 사업비의 70%이하, 현금은 10%이상, 현물은 20% 이하로 작성하세요.(최대 1억원 한도 이내)"라고 적어주세요.
        예시)
        |      |          |           총 사업비(원)             |
        |비 목 | 집행 계획 |정부지원 사업비|자기부담사업비 | 합계 |
        |      |          |              | 현 금 | 현 물 |      |
        |------|----------|--------------|-------|------|-------|
        |개발비|           | 95,000,000  |       |       |      |
        |       합 계      |             |       |       |      |
        - 정부지원사업비가 총 사업비의 규정을 초과하지 않도록 주의하세요.

    - 기타 자금 필요성 및 조달계획: 
        - 사업화 자금 외에 추가자금(투자유치 등) 조달에 대한 필요성과 세부 조달방안을 작성하세요."""

## 팀구성(Team)      
template_test4 = """
4.팀구성(Team)

    4-1. 대표자&팀원 현황 및 보유역량
        1) 대표자 현황 및 역량
            - 대표자만 적어주세요.
            - 예시) 유사 경험, 지원사업 수행 이력, 관련 이수 현황, 관련 수상 실적 등을 작성하세요.
            - 사용자가 구체적으로 작성할 수 있도록 "예시"들을 작성해주세요.
        
            <팀 구성 예정>
            예시) 표로 작성해주세요.("예시"라는 문구를 보여주세요.)
            |순번|직위|담당 업무|보유역량(경력 및 학력 등)|구성 상태|
        
        2) 협약 기간 내 인력 고용 및 활용 계획:
            ("예시"라는 문구를 보여주세요.)
            - 협약기간 내 채용 예정인 추가인력에 대해서만 작성하세요.
            - 채용 예정이 없는 기업의 경우, 대표자의 역량, 보유 장비, 시설 등을 중심으로 작성하세요.
        
            <추가 고용 및 활용 계획>
            예시) 표로 작성해주세요.("예시"라는 문구를 보여주세요.)
            |순번|주요 담당 업무|요구역량(경력 및 학력 등)|채용 시기|
        
        3) 외부 협력 현황 및 활용방안
            - 제품 및 서비스와 관련하여 협력(또는 예정)인 파트너, 협력 기간(기업)이 보유하고 있는 역량과 주요 협업 내용을 작성하세요.
            예시) 표로 작성해주세요.("예시"라는 문구를 보여주세요.)
            |순번|파트너명|보유역량|협업방안|협력 시기|
            
    4-2. 중장기적 ESG 경영 도입계획
        - 재무적 성과 이외에 사회적 가치, 환경 보호, 지역 활성화 등의 구체적인 실천 계획을 작성하세요.
        - 실행 전략: 각 목표를 달성하기 위한 구체적인 실행 전략을 작성하세요.
        - 성과 측정: 각 전략의 성과를 측정할 수 있는 지표와 방법을 제시하세요.
        - 지속 가능성: 도입 계획의 지속 가능성을 보장하기 위한 방안을 작성하세요.
        - 파트너십: 사회적 가치를 도입하기 위해 협력할 수 있는 외부 파트너 또는 기관을 제시하세요."""

In [ ]:
prompt1 = f"""
당신은 예비창업자를 위한 사업계획서 전문가입니다.  
논리적이고 설득력 있는 사업계획서를 작성하세요.  
문장은 간결하게, 핵심을 강조하며, 투자자 및 심사위원이 주목할 수 있도록 수치, 데이터, 시장 반응을 명확히 표현하세요.
이 작업은 창업자의 생계와 미래가 걸린 **절대적으로 중요한 프로젝트**입니다.
작성된 사업계획서의 완성도가 곧 **합격과 실패를 결정**하며, 이를 통해 창업자는 사업을 현실화하고 생존할 기회를 얻게 됩니다.
초안을 **작성할 때는 {loaded_data['문제인식']}내용 기반으로만** 쓰며, **{template_test1}**에 내용들을 순서에 맞게 참고해주세요.
양식은 아래 제공하는 제공하는 템플릿 양식을 참고하여 작성해주세요.

제공하는 템플릿 양식 : {template}

**반드시 지켜야 할 출력 규칙**
    1. 절대 #, * 기호 및 마크다운 스타일 절대 금지
    2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
    3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함. (ex. 출처: 통계청)
"""
completion1 = client.chat.completions.create(model='gpt-4o',
                                            messages=[{'role':'user', 'content':prompt1}],
                                            temperature=0.7,
                                            presence_penalty=0.8,
                                            frequency_penalty=0.5)
print(completion1.choices[0].message.content)

1. 문제인식(Problem)

1-1. 창업아이템 배경 및 필요성
    1) 농산물 유통 구조의 문제점
        - 현재 농산물 유통 시장은 복잡한 중간 과정을 포함하고 있어 생산자와 소비자가 모두 높은 비용 부담을 겪고 있습니다.
        - 비효율적인 가격 책정으로 인해 생산자는 적절한 수익을 확보하지 못하고, 소비자는 높은 가격에 구매하는 상황입니다.
        - 새로운 기술과 데이터 분석을 통한 효율적인 가격 책정이 필수적입니다.

    2) 시장 변화 및 고객 니즈
        - 온라인 플랫폼의 증가로 농산물 판매 방식이 변화하고 있으며, 이에 맞는 새로운 판매 전략이 필요합니다.
        - 소비자들은 더 합리적이고 투명한 가격 정보를 원하며, 이는 데이터 기반의 접근법으로 해결할 수 있습니다.
        - 정책 변화로 인한 규제 완화는 온라인 판매자의 진입 장벽을 낮추고 있습니다.

    3) 목표 및 해결 방안
        - 이 사업은 데이터 기반 솔루션을 통해 농산물 유통의 비효율성을 줄이고자 합니다.
        - 생산자와 소비자 간 직거래를 활성화하여 중간 마진을 최소화하는 것을 목표로 합니다.
        - 이를 통해 모든 이해관계자가 혜택을 누릴 수 있는 건전한 시장 형성을 이루고자 합니다.

1-2. 창업아이템 목표시장(고객) 설정 및 요구사항 분석

    1) 타깃 고객 정의
        - 주요 타깃 고객은 온라인 플랫폼에서 활동하는 농산물 판매자로 설정됩니다.
        - 이들은 주로 소규모 농가 또는 개인 판매자로 구성되어 있으며, 효율적인 가격 책정과 판매 전략에 대한 니즈가 큽니다.
        
    2) 시장 규모 및 전망
        - 국내 온라인 농산물 시장 규모는 연평균 15% 이상 성장하고 있으며, 2025년까지 약 10조 원 규모로 확대될 것으로 예상됩니다. (출처: 한국농수산식품유통공사)
        - 경쟁 강도는 상대적으로 낮으나, 점차 증가

In [ ]:
prompt2 = f"""
당신은 예비창업자를 위한 사업계획서 전문가입니다.  
논리적이고 설득력 있는 사업계획서를 작성하세요.  
문장은 간결하게, 핵심을 강조하며, 투자자 및 심사위원이 주목할 수 있도록 수치, 데이터, 시장 반응을 명확히 표현하세요.
이 작업은 창업자의 생계와 미래가 걸린 **절대적으로 중요한 프로젝트**입니다.
작성된 사업계획서의 완성도가 곧 **합격과 실패를 결정**하며, 이를 통해 창업자는 사업을 현실화하고 생존할 기회를 얻게 됩니다.
초안을 작성할 때는 {loaded_data['실현가능성']}내용 기반으로만** 쓰며, **{template_test2}**에 내용들을 순서에 맞게 참고해주세요.
양식은 아래 제공하는 템플릿 양식을 참고하여 작성해주세요.

제공하는 템플릿 양식 : {template}

**반드시 지켜야 할 출력 규칙**
    1. 절대 #, * 기호 및 마크다운 스타일 절대 금지
    2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
    3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함. (ex. 출처: 통계청)
"""
completion2 = client.chat.completions.create(model='gpt-4o',
                                        messages=[{'role':'user', 'content':prompt2}],
                                        temperature=0.7,
                                        presence_penalty=0.8,
                                        frequency_penalty=0.5)
print(completion2.choices[0].message.content)

2. 실현가능성(Solution)

2-1. 창업 아이템 현황 및 준비 정도

1) 제품·서비스의 필요성
   - 농산물 온라인 판매자는 효율적인 가격 책정과 시간 단축의 필요성을 느끼고 있습니다.
   - 2023년 12월부터 문제 정의 및 가설 검증을 위해 설문조사와 심층 인터뷰를 두 차례 진행하였습니다.
   - 시장 조사를 기반으로 유저 플로우와 정보구조도를 통해 전체적인 기획을 완료했습니다.

2) 아이템 개발단계별 현황
   - 현재 와이어프레임, 프로토타입, 사용성 테스트를 통해 UI/UX 개선 중입니다.
   - 목표 시장과 고객의 반응은 긍정적이며, 빠른 서비스 가치 인식을 목표로 하고 있습니다.
   - 주요 사업화 성과는 초기 고객들의 피드백을 통해 기능 개선 점검이 이루어지고 있는 단계입니다.

2-2. 창업 아이템 실현 및 구체화 방안

1) 협약 기간 내 실현 및 개선 방안
   - 품목별, 판매 단위별 평균가 추이와 현황 확인 기능을 개발할 예정입니다.
   - 취득한 데이터를 독자적으로 가공하여 유의미한 정보를 고객에게 제공할 계획입니다.

2) 목표시장 요구사항 분석 및 개선점
   - 사용자 피드백에 따른 핵심 기능 및 성능, 디자인 개선 사항을 지속적으로 업데이트합니다.
   - 경쟁사의 서비스와 비교해 가격 책정 효율성과 데이터 활용도에서 차별성을 확보할 것입니다.

3) 경쟁력 확보 방안
   - 자사의 데이터 분석 역량을 기반으로 독창적이고 유용한 정보를 제공하여 시장 내 경쟁력을 강화합니다.
    
4) 기술 보호 계획
    - 독자적으로 수집하고 가공한 데이터에 대한 법적 보호 장치를 마련하여 기술 자산을 보호할 것입니다.


In [ ]:
prompt3 = f"""
당신은 예비창업자를 위한 사업계획서 전문가입니다.  
논리적이고 설득력 있는 사업계획서를 작성하세요.  
문장은 간결하게, 핵심을 강조하며, 투자자 및 심사위원이 주목할 수 있도록 수치, 데이터, 시장 반응을 명확히 표현하세요.
이 작업은 창업자의 생계와 미래가 걸린 **절대적으로 중요한 프로젝트**입니다.
작성된 사업계획서의 완성도가 곧 **합격과 실패를 결정**하며, 이를 통해 창업자는 사업을 현실화하고 생존할 기회를 얻게 됩니다.
초안을 **작성할 때는 {loaded_data['성장전략']}내용 기반으로만** 쓰며, **{template_test3}**에 있는 내용들을 순서에 맞게 참고해주세요.
양식은 아래 제공하는 제공하는 템플릿 양식을 참고하여 작성해주세요.

제공하는 템플릿 양식 : {template}

**반드시 지켜야 할 출력 규칙**
    1. 절대 #, * 기호 및 마크다운 스타일 절대 금지
    2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
    3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함. (ex. 출처: 통계청)
"""
completion3 = client.chat.completions.create(model='gpt-4o',
                                        messages=[{'role':'user', 'content':prompt3}],
                                        temperature=0.7,
                                        presence_penalty=0.8,
                                        frequency_penalty=0.5)
print(completion3.choices[0].message.content)

1. 성장전략(Scale-up)

1-1. 비즈니스 모델 및 사업화 추진 성과

    1) 수익 모델
        - 초기 단계에서는 부분서비스 유료화를 통해 농산물 위탁 판매자 시장에 진입.
        - 다양한 이커머스 플랫폼의 데이터를 분석하여 서비스 고도화.
        - 중기적으로 획득한 사용자 데이터베이스를 활용하여 B2B 서브시장 확장.
    
    2) 목표시장 진입 현황 및 추진성과
        - 현재, 국내 20~30대 농산물 위탁 판매자를 주요 고객층으로 설정.
        - 출시 후 첫 해 매출 약 5억 원 기록. (출처: 자사 내부 데이터)
        - 사용자가 꾸준히 증가하며 앱 다운로드 수는 10만 건 달성. (출처: 구글 플레이 스토어)

1-2. 아이템 시장 확대 등 사업화 전략

    1) 국내외 시장 확대 전략
        - 세부 고객 분석을 통해 맞춤형 마케팅 전략 수립 및 실행.
        - 경쟁사의 서비스와 차별화된 빅데이터 기반 액션 가이드 제공으로 경쟁 우위 확보.
    
    2) 목표시장 진출 및 확대 방안
        - 초기 이용자 확보를 위한 무료 체험 기간 운영 및 추천인 프로모션 진행.
        - 해외 플랫폼과 협력하여 글로벌 시장에서의 입지 강화.

    3) 협약기간 내 달성하고자 하는 성과
        - 매출: 연간 매출 10억 원 목표.
        - 투자: 총 5억 원 이상의 외부 투자 유치 계획.

1-3. 사업추진 일정 및 자금운용 계획

    1) 사업 전체 로드맵(일정)
        
       |구분|추진 내용|추진 기간|세부 내용|
       |---|---|---|---|
       |1|시제품 제작|2023년 2월 - 2023년 4월|시제품 설계|
       |2|시장 조사 및 초기 마케팅 활동|2023년 4월 - 2023년 6월|SNS 캠페인, 온라인 광고 진행|

    <사업추진 일정(전체 사업단계)>
   

In [ ]:
prompt4 = f"""
당신은 예비창업자를 위한 사업계획서 전문가입니다.  
논리적이고 설득력 있는 사업계획서를 작성하세요.  
문장은 간결하게, 핵심을 강조하며, 투자자 및 심사위원이 주목할 수 있도록 수치, 데이터, 시장 반응을 명확히 표현하세요.
이 작업은 창업자의 생계와 미래가 걸린 **절대적으로 중요한 프로젝트**입니다.
작성된 사업계획서의 완성도가 곧 **합격과 실패를 결정**하며, 이를 통해 창업자는 사업을 현실화하고 생존할 기회를 얻게 됩니다.
초안을 **작성할 때는 {loaded_data['팀 구성']}내용을 기반으로** 쓰며, **{template_test4}**에 내용들을 순서에 맞게 참고해주세요.
양식은 아래 제공하는 제공하는 템플릿 양식을 참고하여 작성해주세요.

제공하는 템플릿 양식 : {template}

**반드시 지켜야 할 출력 규칙**
    1. 절대 #, * 기호 및 마크다운 스타일 절대 금지
    2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
    3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함. (ex. 출처: 통계청)
"""
completion4 = client.chat.completions.create(model='gpt-4o',
                                        messages=[{'role':'user', 'content':prompt4}],
                                        temperature=0.7,
                                        presence_penalty=0.8,
                                        frequency_penalty=0.5)
print(completion4.choices[0].message.content)

4. 팀 구성(Team)

4-1. 대표자&팀원 현황 및 보유역량

1) 대표자 현황 및 역량
    - 이재호: 경영학 석사 학위 소지, 10년 이상의 사업 운영 경험 보유.
    - 주요 경력: 국내 대기업에서 프로젝트 매니저로 근무하며 다수의 성공적인 프로젝트를 수행.
    - 교육 이수: 리더십 개발 프로그램 수료.
    - 기타 수상 실적: 2020년 우수 창업인 상 수상.

<팀 구성 예정>
|순번|직위|담당 업무|보유역량(경력 및 학력 등)|구성 상태|
|---|---|---|---|---|
|1|대표이사|전략 기획 및 전체 운영 관리|경영학 석사, 10년 이상 경력|구성 완료|

2) 협약 기간 내 인력 고용 및 활용 계획:
    - "추가 인력 채용 계획은 현재 없으며, 기존의 장비와 시설을 최대한 활용하여 운영할 예정입니다."
    
<추가 고용 및 활용 계획>
|순번|주요 담당 업무|요구역량(경력 및 학력 등)|채용 시기|
|-|-|-|-|

3) 외부 협력 현황 및 활용방안
    - 제품 개발과 서비스 제공을 위해 여러 파트너와 협업 중이며, 지속적인 관계 유지 예정.

예시)
|순번|파트너명|보유역량|협업방안|협력 시기|
|-|-|-|-|-|
  
4-2. 중장기적 ESG 경영 도입계획

1) 사회적 가치 실현
    - 지역사회 발전을 위한 프로그램 지원과 일자리 창출 목표 설정.
    
2) 환경 보호 전략
    - 친환경 자재 사용 비율 확대와 에너지 효율 개선 추진.
    
3) 성과 측정 지표 제시
    - 신재생 에너지 사용 비율 증가율, 지역사회 참여도 등으로 성과 평가.

4) 지속 가능성 방안
    - 지속 가능한 공급망 구축 및 지역 커뮤니티와의 긴밀한 협력을 통해 안정성 확보.

5) 파트너십 구축 방안
    - 환경 단체, 지역 NGO 등과의 협력을 통해 공동 프로젝트 진행 계획.

각 항목에서 명확하고 구체적인 실행 계획을 제시함으로써 투자자들에게 사업의 잠재성과 사회적 책임 의식을 강조하였습

In [15]:
# 생성된 사업계획서 초안 출력
사업계획서_초안1 = completion1.choices[0].message.content
사업계획서_초안1 = 사업계획서_초안1.replace("### ", "")  # 자동으로 붙은 '### '을 제거

사업계획서_초안2 = completion2.choices[0].message.content
사업계획서_초안2 = 사업계획서_초안2.replace("### ", "")

사업계획서_초안3 = completion3.choices[0].message.content
사업계획서_초안3 = 사업계획서_초안3.replace("### ", "")

사업계획서_초안4 = completion4.choices[0].message.content
사업계획서_초안4 = 사업계획서_초안4.replace("### ", "")

In [16]:
ycp_dict = {'초안1': 사업계획서_초안1,
            '초안2': 사업계획서_초안2,
            '초안3': 사업계획서_초안3,
            '초안4': 사업계획서_초안4}

In [ ]:
# 딕셔너리 데이터를 JSON 파일로 저장
with open('business_plan/예비창업패키지_사업계획서.json', 'w', encoding='utf-8') as file:
    json.dump(ycp_dict, file, ensure_ascii=False, indent=4)

In [ ]:
# 예비창업패키지
# 초기창업패키지

# 문서 생성
doc = Document()

# 정규식을 사용하여 각 초안을 띄어쓰기 기준으로 나눔
for _, value in ycp_dict.items():
    re_text = re.split(r'\n\n', value)

    for idx in re_text:

        if '|' in idx:
            text = idx.split('|')
            doc.add_paragraph(text[0])

            data = text[1:-1]

            # '---'가 포함된 항목 제거 및 공백 정리
            filtered_data = [item.strip() for item in data if '-' not in item and item]

            # 데이터를 표 형식(2D 리스트)으로 변환
            table_data = []
            row = []

            for item in filtered_data:
                if item == "":  # 빈 문자열(\n 변환)일 경우 새로운 행 시작
                    if row:  # 현재까지 저장된 행이 있다면 추가
                        table_data.append(row)
                        row = []
                else:
                    row.append(item)

            # 마지막 행 추가 (루프 종료 후 남은 데이터 처리)
            if row:
                table_data.append(row)

            # 첫 번째 행을 자동으로 컬럼명으로 설정
            if table_data:
                headers = table_data.pop(0)  # 첫 번째 행을 컬럼명으로 설정
                table_data.insert(0, headers)  # 다시 삽입 (명확하게 구분하기 위함)


            # 표 생성 (행 개수는 데이터 행 개수, 열 개수는 첫 번째 행의 열 개수 기준)
            table = doc.add_table(rows=len(table_data), cols=len(table_data[0]))

            # 표 스타일 적용 (선택 사항: 표 테두리 추가)
            table.style = "Table Grid"

            # 표 데이터 삽입
            for row_idx, row_data in enumerate(table_data):
                for col_idx, cell_data in enumerate(row_data):
                    cell = table.cell(row_idx, col_idx)
                    cell.text = str(cell_data)  # 데이터 삽입
                    
                    # 첫 번째 행(컬럼명)은 굵게 표시
                    if row_idx == 0:
                        for paragraph in cell.paragraphs:
                            for run in paragraph.runs:
                                run.bold = True  # 글씨 굵게
                                run.font.size = Pt(12)  # 글씨 크기 조정 (선택 사항)

            doc.add_paragraph(text[-1])

        else:
            doc.add_paragraph(idx)

    doc.add_paragraph('\n\n') # 나눴던 구분자를 다시 집어넣음
    
# 파일 저장
doc.save("business_plan/예비창업패키지_초안.docx")

print("Word 파일이 성공적으로 생성되었습니다!")

In [ ]:
convert("business_plan/예비창업패키지_초안.docx", "business_plan/예비창업패키지_초안.pdf")